## Домашнее задание по блоку 15.03.2021. Функции и работа с данными
### Тихонова Инна

In [1]:
import pandas as pd

**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class.

***Допущения при решении задачи:***
 - *под оценкой понимается средняя оценка за фильм по всем проставленным оценкам*
 - *высказывание "оценка 4.5 и 5" понимается как "оценка между 4.5 и 5"*
 - *для интвервала оценок между 4 и 4.5 добавлено значение "undefined"*

In [2]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
def class_by_rating(movieId, ratings):
    
    mean_rate = ratings.query('movieId == @movieId')['rating'].mean()
    
    if mean_rate <= 2:
        return 'низкий рейтинг'
    elif mean_rate <= 4:
        return 'средний рейтинг'
    elif mean_rate <= 4.5:
        return 'undefined'
    else:
        return 'высокий рейтинг'

In [5]:
movies['class'] = movies['movieId'].apply(class_by_rating, ratings = ratings)

In [6]:
movies.groupby('class').count()

,movieId,title,genres
class,,,
undefined,909,909,909
высокий рейтинг,422,422,422
низкий рейтинг,973,973,973
средний рейтинг,6821,6821,6821


**Задание 2**

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т.е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [7]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [8]:
keywords = pd.read_csv('keywords.csv')
keywords.tail()

,keyword,shows
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688
99999,ггму,3630


In [9]:
def find_city(keyword, cities):
    
    for city in cities:
        if city in keyword:
            return True
    return False    
                

def set_region(keyword, geo_data):
    
    for region, cities in geo_data.items():
        if find_city(keyword, cities):
            return region
    return 'undefined'

In [10]:
keywords['region'] = keywords['keyword'].apply(set_region, geo_data = geo_data)

In [11]:
keywords.groupby('region').count()

,keyword,shows
region,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,266,266
Центр,387,387


**Задание 3 (бонусное)**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т.е. проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
 - для каждой строки пройдите по всем годам списка years
 - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
 - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [12]:
ratings = pd.read_csv("ratings.dat", sep='::', engine='python',
                     names = ['UserID', 'MovieID', 'Rating', 'Timestamp'])
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [13]:
movies = pd.read_csv("movies.dat", sep='::', engine='python',
                    names = ['MovieID', 'Title', 'Genres'])
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


1. В переменную years запишем список из всех годов с 1950 по 2010

In [14]:
years = [x for x in range(1950, 2011)]

2. Напишем функцию production_year, которая каждой строке из названия фильма выставляет год выпуска:
 - для каждой строки пройдите по всем годам списка years
 - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
 - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

In [15]:
def production_year(title, years):
    
    for year in years:
        if str(year) in title:
            return year
    return 1900    

3. Запишем год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

In [16]:
movies['year'] = movies['Title'].apply(production_year, years = years)

In [17]:
movies.head()

,MovieID,Title,Genres,year
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


4. Посчитаем средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируем результат по убыванию рейтинга

In [18]:
movies_ratings = movies.merge(

    (ratings
     .groupby('MovieID')
     .mean()
    ),
    on='MovieID'

)[['year', 'Rating']]

movies_ratings.head()

,year,Rating
0,1995,4.146846
1,1995,3.201141
2,1995,3.016736
3,1995,2.729412
4,1995,3.006757


In [19]:
movies_ratings.groupby('year')['Rating'].mean().sort_values(ascending=False).head()

year
1962    3.803059
1951    3.795416
1950    3.758235
1954    3.748782
1966    3.731684
Name: Rating, dtype: float64

In [20]:
movies_ratings.groupby('year')['Rating'].mean().sort_values(ascending=False).tail()

year
1990    3.095630
1987    3.079078
1996    3.073662
1988    3.069941
2000    3.050624
Name: Rating, dtype: float64